# 05: Training...


In [6]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras import layers
from rxwgan.metrics import sp_metric
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.python.client import device_lib
import json,os

#print(device_lib.list_local_devices())
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
path = '/home/jodafons/public/bric_data/Shenzhen/raw/Shenzhen_table_from_raw.csv'

In [3]:
dataframe = pd.read_csv(path)
target = dataframe.target.values

# kfold
kf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 512)

# image generator
datagen = ImageDataGenerator( rescale=1./255 )

batch_size = 32

In [4]:
def create_model(height, width):
    
    ip = layers.Input(shape=( height,width,1))
    y = layers.Conv2D(128, (5,5), strides=(2,2), padding='same', kernel_initializer='he_uniform', data_format='channels_last', input_shape=(height,width,1))(ip)
    #y = layers.BatchNormalization()(y)
    y = layers.Activation('relu')(y)
    y = layers.Dropout(rate=0.3, seed=1)(y)
    # Output (None, 3^2*2^3, 64)
    y = layers.Conv2D(64, (5,5), strides=(2,2), padding='same', kernel_initializer='he_uniform')(y)
    #y = layers.BatchNormalization()(y)
    y = layers.Activation('relu')(y)
    y = layers.Dropout(rate=0.3, seed=1)(y)
      # Output (None, 3^2*2^3, 64)
    y = layers.Conv2D(32, (5,5), strides=(2,2), padding='same', kernel_initializer='he_uniform')(y)
    #y = layers.BatchNormalization()(y)
    y = layers.Activation('relu')(y)
    y = layers.Dropout(rate=0.3, seed=1)(y)
    # Output (None, 3^2*2, 128)
    y = layers.Flatten()(y)
    # Output (None, 3*256)
    y = layers.Dense(64, activation='relu')(y)
    out = layers.Dense(1, activation='sigmoid')(y)
    # Output (None, 1)
    model = tf.keras.Model(ip, out)
    model.compile(optimizer="adam",loss='binary_crossentropy',metrics=['acc',sp_metric])
    #model.summary()
    return model

## Train Model:

Reshape each image to 128 X 128 and rescale all to gray (1/255). Reading all from tuberculoses rx images

In [10]:

for fold, (train_idx, val_idx) in enumerate(kf.split(dataframe , target)):
    
    output_path = 'output/discriminator/fold_%d'%fold
    os.makedirs(output_path, exist_ok=True)
    training_data   = dataframe.iloc[train_idx]
    validation_data = dataframe.iloc[val_idx]
    #tf.config.run_functions_eagerly
    model = create_model(128,128)
    
    train_generator = datagen.flow_from_dataframe(training_data, directory = None,
                                                  x_col = 'raw_image_path', 
                                                  y_col = 'target',
                                                  batch_size = batch_size,
                                                  target_size = (128,128), 
                                                  class_mode = 'raw', 
                                                  shuffle = False,
                                                  color_mode = 'grayscale')
    
    val_generator   = datagen.flow_from_dataframe(validation_data, directory = None,
                                                  x_col = 'raw_image_path', 
                                                  y_col = 'target',
                                                  batch_size = batch_size,
                                                  class_mode = 'raw',
                                                  target_size = (128,128),
                                                  shuffle = False,
                                                  color_mode = 'grayscale')
    
    
    stop = EarlyStopping(monitor='val_sp', mode='max', verbose=1, patience=25, restore_best_weights=True)
    
    #classes = (0,1)
    #y_train = training_data.target.values
    #weights = compute_class_weight('balanced',classes,y_train)
    #class_weights = {cl : weights[idx] for idx, cl in enumerate(classes)}
    #print(class_weights)
    #sample_weight = np.ones_like(y_train, dtype=np.float32)
    #sample_weight[y_train==1] = weights[1]
    #sample_weight[y_train==0] = weights[0] 
    
    history = model.fit_generator(train_generator,
                validation_data=val_generator,
                epochs=1000,
                callbacks=[stop],
                #sample_weight=sample_weight,
                shuffle=True,
                verbose=1).history
    
    model.save(output_path+'/discr_trained.h5')
    with open(output_path+'/history.json', 'w') as handle:
        json.dump(history, handle,indent=4)

    
    

Found 595 validated image filenames.
Found 67 validated image filenames.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/1000
19/19 [==============================] - 66s 3s/step - loss: 0.9066 - acc: 0.5092 - sp: 0.5141 - val_loss: 0.6765 - val_acc: 0.7612 - val_sp: 0.8325
Epoch 2/1000
19/19 [==============================] - 30s 2s/step - loss: 0.6274 - acc: 0.6588 - sp: 0.6696 - val_loss: 0.5893 - val_acc: 0.6269 - val_sp: 0.8957
Epoch 3/1000
19/19 [==============================] - 30s 2s/step - loss: 0.5759 - acc: 0.7294 - sp: 0.7294 - val_loss: 0.5862 - val_acc: 0.8657 - val_sp: 0.8957
Epoch 4/1000
19/19 [==============================] - 30s 2s/step - loss: 0.5857 - acc: 0.7227 - sp: 0.7273 - val_loss: 0.4279 - val_acc: 0.8657 - val_sp: 0.8957
Epoch 5/1000
19/19 [==============================] - 30s 2s/step - loss: 0.5049 - acc: 0.7882 - sp: 0.8052 - val_loss: 0.3958 - val_acc: 0.8507 - val_sp: 0.8808
Epoch 6/1000
19/19 [==============================] - 30s 2s/step - loss: 0.4776 - acc: 0.8050 - sp: 0.8063 - val_loss: 0.4074 - val_acc: 0.8358 - val_sp: 0.8808
Epoch 7/1000
19/19 [========